In [1]:
# make df by def
import glob
import os
import pandas as pd
def get_data(dn):
    data = {
        "news":[],
        "ans":[]
    }
    dirs = glob.glob(os.path.join(dn, "*"))# /chinese_news_trans\\交通..etc *輸出所有資料夾/ glob return list 重要處理大量資料用
    for d in dirs:
        pat = os.path.join(d, "*.txt") #/chinese_news_trans\交通..etc\*.txt/ *path組合: d+*.txt
        fs = glob.glob(pat)# a list with all txt's path
        for fp in fs:
            with open(fp, "r", encoding="utf-8") as f:
                news = f.read()
            ans = os.path.split(d)[-1]# get title *交通　體育..
            data["news"].append(news)
            data["ans"].append(ans)
    df = pd.DataFrame(data, columns=["news", "ans"])
    return df

In [2]:
# make df with fit
train_df = get_data("chinese_news_trans/chinese_news_trans")
test_df = get_data("chinese_news_test/chinese_news_test")
test_df

news ans
0    日月光華 -- Traffic_Info精華區文章閱讀\n-----------------...  交通
1    日月光華 -- Traffic_Info精華區文章閱讀\n-----------------...  交通
2    日月光華 -- Traffic_Info精華區文章閱讀\n-----------------...  交通
3    三趟火車停開 乘客可全額退票 \n瀏覽次數：1180 \n    昨日，來自鐵路部門的消息說...  交通
4    日月光華 -- Traffic_Info精華區文章閱讀\n-----------------...  交通
..                                                 ...  ..
96   最優秀選手無緣亞運會健美賽\n \n　 \n    健美在亞洲運動會上是“新生兒”——韓國釜...  體育
97    \n各國記者眼中的羽毛球世錦賽\n \n\n-----------------------...  體育
98   \n友好運動會第五天 東道主選手大顯神威\n2001年09月03日02:51:40 新華社 ...  體育
99   不靠技術比運氣\n\n　　第二屆奧運會在巴黎舉行，同時這裡也正在舉行國際博覽會，東道主把一些...  體育
100  帆 板 運 動 簡 介（二）\n\n　　我國在79年由國家體委青島航海運動學校成功試製出第一...  體育

[101 rows x 2 columns]

In [3]:
# adj. df[ans]
# ans replaced str with int 0,1,2.. for original df
# df - unique, value_counts
kind = train_df['ans'].unique() # 取ans不重複
# enumerate - 自動加編號
trans = {k:i for i, k in enumerate(kind)}
reverse_trans = {i:k for i, k in enumerate(kind)}
y_train = train_df["ans"].replace(trans)
y_test = test_df["ans"].replace(trans)
y_test


0      0
1      0
2      0
3      0
4      0
      ..
96     9
97     9
98     9
99     9
100    9
Name: ans, Length: 101, dtype: int64

In [4]:
import os
from urllib.request import urlretrieve
DICT_PATH = "dict.txt.big"
if not os.path.exists(DICT_PATH):
    print("字典不存在")
    url = "https://github.com/fxsjy/jieba/raw/master/extra_dict/dict.txt.big"
    urlretrieve(url, DICT_PATH)

字典不存在


In [5]:
# adj. df['news']
import jieba
jieba.set_dictionary(DICT_PATH) # import大辭典
def newscut(news): #創分詞函式
    s = " ".join(jieba.cut(news))
    return s.replace("\r", "").replace("\n", "")
x_train_raw = train_df["news"].apply(newscut)#分詞新聞內容
x_test_raw = test_df["news"].apply(newscut)
x_test_raw

Building prefix dict from C:\Users\user\PycharmProjects\MLDemo\dict.txt.big ...
Dumping model to file cache C:\Users\user\AppData\Local\Temp\jieba.u8b0aede1d05d994ba4294f10e50c49e7.cache
Loading model cost 1.376 seconds.
Prefix dict has been built successfully.


0      日月 光華   --   Traffic _ Info 精華區 文章 閱讀  -------...
1      日月 光華   --   Traffic _ Info 精華區 文章 閱讀  -------...
2      日月 光華   --   Traffic _ Info 精華區 文章 閱讀  -------...
3      三趟 火車 停開   乘客 可 全額 退票    瀏覽 次數 ： 1180         ...
4      日月 光華   --   Traffic _ Info 精華區 文章 閱讀  -------...
                             ...                        
96     最 優秀 選手 無緣 亞運會 健美 賽     　            健美 在 亞洲 運...
97        各國 記者 眼中 的 羽毛球 世錦賽      -------------------...
98      友好 運動會 第五天   東道主 選手 大顯神威  2001 年 09 月 03 日 02...
99     不靠 技術 比 運氣   　 　 第二屆 奧運會 在 巴黎 舉行 ， 同時 這裡 也 正在 ...
100    帆   板   運   動   簡   介 （ 二 ）   　 　 我國 在 79 年 由 ...
Name: news, Length: 101, dtype: object

In [6]:
# make df2 at vec
#創詞向量　餵訓練&測試題目
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer()
x_train = vec.fit_transform(x_train_raw)
x_test = vec.transform(x_test_raw) #test without "fit"
x_test

<101x76399 sparse matrix of type '<class 'numpy.int64'>'
	with 27152 stored elements in Compressed Sparse Row format>

In [7]:
# set model + train model
#創單純貝式模型＋訓練　
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(x_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [10]:
# predict
from sklearn.metrics import accuracy_score
pre = clf.predict(x_test)
accuracy_score(pre, y_test)

1.0

In [11]:
from sklearn.metrics import confusion_matrix
mat = confusion_matrix(y_test, pre)
col = ["{}(預測)".format(k) for k in kind]
row = ["{}(答案)".format(k) for k in kind]
pd.DataFrame(mat, columns=col, index=row)

交通(預測)  政治(預測)  教育(預測)  環境(預測)  經濟(預測)  藝術(預測)  計算機(預測)  軍事(預測)  \
交通(答案)       10       0       0       0       0       0        0       0   
政治(答案)        0      11       0       0       0       0        0       0   
教育(答案)        0       0      10       0       0       0        0       0   
環境(答案)        0       0       0      10       0       0        0       0   
經濟(答案)        0       0       0       0      10       0        0       0   
藝術(答案)        0       0       0       0       0      10        0       0   
計算機(答案)       0       0       0       0       0       0       10       0   
軍事(答案)        0       0       0       0       0       0        0      10   
醫藥(答案)        0       0       0       0       0       0        0       0   
體育(答案)        0       0       0       0       0       0        0       0   

         醫藥(預測)  體育(預測)  
交通(答案)        0       0  
政治(答案)        0       0  
教育(答案)        0       0  
環境(答案)        0       0  
經濟(答案)        0       0  
藝術(答案)        0       0  
計算機(答案)       0       0  
軍事(答案)        0       0  
醫藥(答案)       10       0  
體育(答案)        0      10

In [13]:
# self-app created
n = input("請輸入新聞:")
test = vec.transform([newscut(n)])
pre = clf.predict(test)[0]
print("應該是:", reverse_trans[pre])
proba = clf.predict_proba(test)[0]
proba = list(enumerate(proba))
# proba
for i, prob in sorted(proba, key=lambda x:x[1], reverse=True):
    print(reverse_trans[i], ":", round(prob*100), "%")

請輸入新聞:Sony Interactive Entertainment在官方公告中提到，有鑑人們為防止肺炎疫情擴散安分守己地待在家中，為了感謝大家保持社交距離，並盡量滿足大家「Play At Home」（在家遊玩）的需求，因此決定推出包含兩部分內容的「Play At Home」活動，第一是提供給所有PS4的玩家在限定期間內可免費下載《秘境探險：德瑞克合輯》與《風之旅人》2款遊戲，兌換後即可一直擁有。     《秘境探險：德瑞克合輯(UNCHARTED: The Nathan Drake Collection)》，是將原本由遊戲製作室Naughty Dog創作的冒險遊戲，由Bluepoint Games發揮PS4™主機的性能重製登場。合輯收錄了系列前 3 代《秘境探險：黃金城秘寶》、《秘境探險 2：盜亦有道》、《秘境探險 3：德瑞克的騙局》的遊戲，玩家將扮演奈森・德瑞克，從微不足道的事端開始，到難以想像的發現，踏上橫越地球的驚險旅程。而德瑞克亦會與令人難忘的各式人物相遇，以性命和友情作為賭注，與陰險殘酷的敵人競爭追蹤傳說中的寶藏。目前PS Store售價為NT$590元，並曾作為2020年1月「PlayStation Plus」會員免費下載遊戲。
應該是: 計算機
計算機 : 100.0 %
藝術 : 0.0 %
軍事 : 0.0 %
體育 : 0.0 %
經濟 : 0.0 %
環境 : 0.0 %
教育 : 0.0 %
醫藥 : 0.0 %
交通 : 0.0 %
政治 : 0.0 %


In [ ]:
list(zip([1, 2, 3],
         [7, 8, 9],
         [4, 5, 6]))

In [ ]:
sorted([(3, "a"), (1, "b"), (5, "c")])